### The same of 02dataTraining but with SQL commands

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (
    SparkSession
    .builder
    .master("local[*]")
    .appName("sparkSQL")
    .getOrCreate()
)

24/02/29 17:03:18 WARN Utils: Your hostname, DESKTOP-9M9GKHV resolves to a loopback address: 127.0.1.1; using 172.28.62.164 instead (on interface eth0)
24/02/29 17:03:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/29 17:03:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/29 17:03:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = (
    spark
    .read
    .option("delimiter", ';')
    .option("header", 'true')
    .option("inferSchema", 'true')
    .option("enconding", 'ISO-8859-1')
    .csv(r"./data.test/Preços semestrais - AUTOMOTIVOS_2023.02.csv")
)

In [4]:
df.printSchema()

root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [5]:
df.createOrReplaceTempView('combustiveis')

In [6]:
spark.sql("""
    select `Estado - Sigla`, `Produto`, `Valor de Compra`, `Valor de Venda`, `Unidade de Medida` from combustiveis
""").show(5)

+--------------+----------+---------------+--------------+-----------------+
|Estado - Sigla|   Produto|Valor de Compra|Valor de Venda|Unidade de Medida|
+--------------+----------+---------------+--------------+-----------------+
|            SP|  GASOLINA|           NULL|          4,87|       R$ / litro|
|            SP|DIESEL S10|           NULL|          4,88|       R$ / litro|
|            SP|    ETANOL|           NULL|          3,27|       R$ / litro|
|            AC|  GASOLINA|           NULL|          6,95|       R$ / litro|
|            AC|DIESEL S10|           NULL|          6,85|       R$ / litro|
+--------------+----------+---------------+--------------+-----------------+
only showing top 5 rows



In [7]:
spark.sql("""
    select * from combustiveis where `Valor de Compra` is NOT NULL
""").show()

+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+
|Regiao - Sigla|Estado - Sigla|Municipio|Revenda|CNPJ da Revenda|Nome da Rua|Numero Rua|Complemento|Bairro|Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|Bandeira|
+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+
+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+



In [29]:
view_Precos = spark.sql("""
    select 
        `Estado - Sigla`, 
        `Produto`, 
        regexp_replace(`Valor de Venda`, ",", ".") as `Valor de Venda`, 
        `Unidade de Medida` 
    from combustiveis

""")

In [36]:
view_Precos.createOrReplaceTempView('view_Precos')
view_Precos.show(5)

+--------------+----------+--------------+-----------------+
|Estado - Sigla|   Produto|Valor de Venda|Unidade de Medida|
+--------------+----------+--------------+-----------------+
|            SP|  GASOLINA|          4.87|       R$ / litro|
|            SP|DIESEL S10|          4.88|       R$ / litro|
|            SP|    ETANOL|          3.27|       R$ / litro|
|            AC|  GASOLINA|          6.95|       R$ / litro|
|            AC|DIESEL S10|          6.85|       R$ / litro|
+--------------+----------+--------------+-----------------+
only showing top 5 rows



In [40]:
df_DifPrice = spark.sql("""
    select 
        `Estado - Sigla`, 
        `Produto`, 
        `Valor de Venda`, 
        `Unidade de Medida`,
        MIN(`Valor de Venda`) as minPrice,
        MAX(`Valor de venda`) as maxPrice,
        MAX(`Valor de venda`) - MIN(`Valor de Venda`) as dif
    from view_Precos
    group by all
    order by dif DESC

""")

In [42]:
df_DifPrice.show(5)

+--------------+-------+--------------+-----------------+--------+--------+---+
|Estado - Sigla|Produto|Valor de Venda|Unidade de Medida|minPrice|maxPrice|dif|
+--------------+-------+--------------+-----------------+--------+--------+---+
|            AC| DIESEL|          5.65|       R$ / litro|    5.65|    5.65|0.0|
|            AC| DIESEL|          5.69|       R$ / litro|    5.69|    5.69|0.0|
|            AC| DIESEL|          5.72|       R$ / litro|    5.72|    5.72|0.0|
|            AC| DIESEL|          5.75|       R$ / litro|    5.75|    5.75|0.0|
|            AC| DIESEL|          5.76|       R$ / litro|    5.76|    5.76|0.0|
+--------------+-------+--------------+-----------------+--------+--------+---+
only showing top 5 rows

